In [1]:
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_regression, chi2, VarianceThreshold
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import sympy
from sympy import Matrix
from scipy.linalg import null_space
import itertools

In [3]:
Data = pd.read_excel(r'database.xlsx', index_col=0)

In [4]:
import tqdm
rows_to_drop = []
for i in tqdm.tqdm(range(len(Data))):
    if 'Cu' in Data.index[i] or 'Fe' in Data.index[i] or 'Ni' in Data.index[i] or 'O' in Data.index[i] or Data['Tc'].iloc[i]>50:
        rows_to_drop.append(Data.index[i])

100%|██████████| 16763/16763 [00:00<00:00, 149670.61it/s]


In [5]:
data_reduced = Data.drop(rows_to_drop)
train_df, test_df = train_test_split(data_reduced, test_size = 0.15, random_state = 0)

In [11]:
sample = np.array([data['MagpieData range MeltingT'].values, 
                   data['0-norm'].values, 
                   data['MagpieData mode NdUnfilled'].values]).transpose()

In [43]:
H, edges = np.histogramdd(sample, bins = [10, 6, 10], range = [None, None, None])

In [46]:
edges = [np.array([ 0.,  380.899,  761.798, 1142.697, 1523.596, 1904.495, 2285.394, 2666.293, 3047.192, 3428.091, 3808.99 ]),
                  np.array([1.        , 1.83333333, 2.66666667, 3.5       , 4.33333333, 5.16666667, 6.        ]),
         np.array([0. , 0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2, 8.1, 9. ])
]

In [51]:
def define_system(sample):
    
    H, edges = np.histogramdd(sample, bins = [10, 6, 10], range = [[0, 3808.99], [1., 6.], [0., 9.]])
    n_variables = np.shape(sample)[1]
    alpha_x1 = (edges[0] + (edges[0][1]-edges[0][0])/(2))[:-1]
    alpha_x2 = (edges[1] + (edges[1][1]-edges[1][0])/(2))[:-1]
    alpha_x3 = (edges[2] + (edges[2][1]-edges[2][0])/(2))[:-1]

    combinations = np.array(list(itertools.product(alpha_x1, alpha_x2, alpha_x3)))

    m = np.zeros([int(n_variables + (n_variables**2 - n_variables)/2 + n_variables), len(combinations)]) ## define matrix m

    for i in range(n_variables):                            ## First rows: bins
        m[i] = combinations.transpose()[i]

    variance = []
    for i in range(n_variables):
        for j in range(n_variables):
            if i >= j:
                variance.append( (m[i] - np.mean(sample[:, i])) * (m[j] - np.mean(sample[:, j])) )

    for i in range(len(variance)):                          ## then, variances
        m[i + n_variables] = variance[i]

    D = np.array([np.ones(np.shape(m)[1])])
    E = np.vstack([m, D])
    rho = null_space(D)
    t = null_space(E)

    p = np.zeros([np.shape(m)[0] + 1, np.shape(m)[1]])
    p[0] = np.ones(np.shape(m)[1]) * 1/np.shape(m)[1]

    p = p[0]
    
    return(H, p, m, rho, t)

def correct(N, p, m, rho, t, eps1, eps2, eps3, eps4, eps5, eps6, eps7, eps8, eps9):
    
    A = np.zeros([np.shape(rho)[1], np.shape(rho)[1]])
    b = np.zeros(np.shape(rho)[1])
    R = np.zeros([N, 600])
    
    for k in range(N):
        for i in range(np.shape(t)[1]):
            for j in range(np.shape(rho)[1]):
                A[i][j] = np.dot(t[:, i], np.dot(np.diag(-1/p), rho[:, j]))

        for j in range(np.shape(rho)[1]):
            
            A[-9][j] = np.dot(rho[:, j], m[0])
            A[-8][j] = np.dot(rho[:, j], m[1])
            A[-7][j] = np.dot(rho[:, j], m[2])
            A[-6][j] = np.dot(rho[:, j], m[3])
            A[-5][j] = np.dot(rho[:, j], m[4])
            A[-4][j] = np.dot(rho[:, j], m[5])
            A[-3][j] = np.dot(rho[:, j], m[6])
            A[-2][j] = np.dot(rho[:, j], m[7])
            A[-1][j] = np.dot(rho[:, j], m[8])

        for i in range(np.shape(t)[1]):
            b[i] = np.dot((1 + np.log(p)), t[:, i])
        
        b[-9] = eps1
        b[-8] = eps2
        b[-7] = eps3
        b[-6] = eps4   
        b[-5] = eps5
        b[-4] = eps6
        b[-3] = eps7
        b[-2] = eps8
        b[-1] = eps9


        mu = np.linalg.solve(A, b)
        delta_c = np.dot(mu, rho.transpose())

        p = p + delta_c
        R[k] = p
        
    return(R)

def auto_correct(H, p, m, rho, t):
    for i in tqdm.tqdm(range(np.shape(m)[0])):
        
        res = np.dot(m[i], H.flatten()/sum(H.flatten())) - np.dot(m[i], p)
        if abs(res)>=.1:
            eps = res/2
            a = np.zeros(np.shape(m)[0])
            a[i] = eps
            l = correct(2, p, m, rho, t, a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8])
            p = l[-1]
            print(p)
            
    return(p)

In [37]:
import tqdm
rows_to_drop = []
for i in tqdm.tqdm(range(len(Data))):
    if 'Cu' in Data.index[i] or 'Fe' in Data.index[i] or 'Ni' in Data.index[i] or 'O' in Data.index[i] or Data['Tc'].iloc[i]>50:
        rows_to_drop.append(Data.index[i])

100%|██████████| 16763/16763 [00:00<00:00, 158065.15it/s]


In [ ]:
train_df['class'] = 0
for i in range(len(train_df)):
    if train_df['Tc'].iloc[i] >= 15:
        train_df['class'].iloc[i] = 1
        
test_df['class'] = 0
for i in range(len(test_df)):
    if test_df['Tc'].iloc[i] >= 15:
        test_df['class'].iloc[i] = 1

class_data = test_df[['MagpieData range MeltingT', '0-norm', 'MagpieData mode NdUnfilled', 'class']]
class_data[['bin_dimension_1', 'bin_dimension_2', 'bin_dimension_3']] = 0

In [ ]:
J0 = np.zeros([1, 600])
J1 = np.zeros([1, 600])

for i in tqdm.tqdm(range(1)):
    
    data0 = train_df[train_df['Tc']<15]
    data1 = train_df[train_df['Tc']>=15]
    sample0 = np.array([data0['MagpieData range MeltingT'].values, 
                        data0['0-norm'].values, 
                        data0['MagpieData mode NdUnfilled'].values]).transpose()
    sample1 = np.array([data1['MagpieData range MeltingT'].values, 
                        data1['0-norm'].values, 
                        data1['MagpieData mode NdUnfilled'].values]).transpose()
    
    H0, p0, m0, rho0, t0 = define_system(sample0)
    H1, p1, m1, rho1, t1 = define_system(sample1)
    
    J0[i] = auto_correct(H0, p0, m0, rho0, t0)
    J1[i] = auto_correct(H1, p1, m1, rho1, t1)